## Hyperledger Fabric Python SDK

In [22]:
# To start fabric network in the installed fabric-sdk library

# $ HLF_VERSION=1.4.6
# $ docker pull hyperledger/fabric-peer:${HLF_VERSION}
# $ docker pull hyperledger/fabric-orderer:${HLF_VERSION}
# $ docker pull hyperledger/fabric-ca:${HLF_VERSION}
# $ docker pull hyperledger/fabric-ccenv:${HLF_VERSION}
# $ docker-compose -f test/fixtures/docker-compose-2orgs-4peers-tls.yaml up

## 1. Get Credentials

### 1.1 Load the Connection Profile

### Client
* Main interaction handler with end user.
* Client can maintain several channels.
* Creation of a client with empty network profile

In [72]:
from hfc.fabric import Client

cli = Client(net_profile="test/fixtures/network.json") #/home/ghanendra/fabric-sdk-py/test/fixtures/network.json")

print(cli.organizations)  # orgs in the network
print(cli.peers)  # peers in the network
print(cli.orderers)  # orderers in the network
print(cli.CAs)  # ca nodes in the network

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


{'orderer.example.com': <hfc.fabric.organization.Organization object at 0x7fbb68201370>, 'org1.example.com': <hfc.fabric.organization.Organization object at 0x7fbb60630eb0>, 'org2.example.com': <hfc.fabric.organization.Organization object at 0x7fbb681efc40>}
{'peer0.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fbb681ec430>, 'peer1.org1.example.com': <hfc.fabric.peer.Peer object at 0x7fbb681ece80>, 'peer0.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fbb6806fe80>, 'peer1.org2.example.com': <hfc.fabric.peer.Peer object at 0x7fbb68084130>}
{'orderer.example.com': <hfc.fabric.orderer.Orderer object at 0x7fbb681ec460>}
{'ca-org1': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fbb681ec190>, 'ca-org2': <hfc.fabric.certificateAuthority.certificateAuthority object at 0x7fbb681ec400>}


### Sample Network with 4 peers(2 organisations), 1 orderer 2 certificate authority
* Network.json contains the defined hyperledger fabric sample network

## 1.2 Prepare User Id (Optionally)

### SDK need the credential file as a valid network user.

* #### Typically there are two ways: using cryptogen or using Fabric-CA. That will depend on how your network boots up with.
* #### 1.2.1 Using Local Credential
* #### SDK will load the valid credential from local path (the credentail files must be put there in advance).

In [3]:
from hfc.fabric import Client

cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin') # get the admin user from local path

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


### 1.2.1 Hyperledger fabric network information

In [27]:
cli.get_net_info()

{'name': 'sample-network',
 'description': 'Sample network contains 4 peers (2 orgs), 1 orderer and 2 cas for Python SDK testing',
 'version': '0.1',
 'client': {'organization': 'Org1',
  'credentialStore': {'path': '/tmp/hfc-kvs',
   'cryptoStore': {'path': '/tmp/hfc-cvs'},
   'wallet': 'wallet-name'}},
 'organizations': {'orderer.example.com': {'mspid': 'OrdererMSP',
   'orderers': ['orderer.example.com'],
   'certificateAuthorities': ['ca-orderer'],
   'users': {'Admin': {'cert': 'test/fixtures/e2e_cli/crypto-config/ordererOrganizations/example.com/users/Admin@example.com/msp/signcerts/Admin@example.com-cert.pem',
     'private_key': 'test/fixtures/e2e_cli/crypto-config/ordererOrganizations/example.com/users/Admin@example.com/msp/keystore/630e3767a6e1d3c8e646460123d397455103a900efb4d6fb679a9d9c481841fc_sk'}}},
  'org1.example.com': {'mspid': 'Org1MSP',
   'peers': ['peer0.org1.example.com', 'peer1.org1.example.com'],
   'certificateAuthorities': ['ca-org1'],
   'users': {'Admin': {'

## 1.2.2 Get Credentail from Fabric CA

### Here demos how to interact with Fabric CA.

* Enroll into Fabric CA with admin role;
* Register a user user1;
* Enroll with the new user user1 and get local credential;
* Re-enroll the user1;
* Revoke the user1.

To use CA, a CA server must be started. For example,

In [ ]:
#$ docker-compose -f test/fixtures/ca/docker-compose.yml up

In [5]:
from hfc.fabric_ca.caservice import ca_service

casvc = ca_service(target="http://127.0.0.1:7054")
adminEnrollment = casvc.enroll("admin", "adminpw") # now local will have the admin enrollment
secret = adminEnrollment.register("user2") # register a user to ca
user1Enrollment = casvc.enroll("user2", secret) # now local will have the user enrollment
user1ReEnrollment = casvc.reenroll(user1Enrollment) # now local will have the user reenrolled object
RevokedCerts, CRL = adminEnrollment.revoke("user2") # revoke the user if you need

### You can also use the new identity management system:

In [6]:
from hfc.fabric_ca.caservice import ca_service

casvc = ca_service(target="http://127.0.0.1:7054")
identityService = casvc.newIdentityService()

admin = casvc.enroll("admin", "adminpw") # now local will have the admin user
secret = identityService.create(admin, 'foo') # create user foo
res = identityService.getOne('foo', admin) # get user foo
res = identityService.getAll(admin) # get all users
res = identityService.update('foo', admin, maxEnrollments=3, affiliation='.', enrollmentSecret='bar') # update user foo
res = identityService.delete('foo', admin) # delete user foo

# 2. Operate Channels with Fabric Network

### 2.1 Create a new channel and join it

* #### Use sdk to create a new channel and let peers join it.

### Get the organisations and define the organisation admins.

In [73]:
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin') # User instance with the Org1 admin's certs
org2_admin = cli.get_user(org_name='org2.example.com', name='Admin') # User instance with the Org2 admin's certs
orderer_admin = cli.get_user(org_name='orderer.example.com', name='Admin') # User instance with the orderer's certs

### Create a New Channel, the response should be true if succeed

In [74]:
response = await cli.channel_create(
                    orderer='orderer.example.com',
                    channel_name='businesschannel',
                    requestor=org1_admin,
                    config_yaml='test/fixtures/e2e_cli/',
                    channel_profile='TwoOrgsChannel'
                    )

# response = true is returned if the channel is created successfully
print(response)

FABRIC_CFG_PATH set to /home/ghanendra/fabric-sdk-py/test/fixtures/e2e_cli/
Configtx file successfully created in current             directory
{'tx_id': 'b944313bf01d4ecf8be0b1c04e140025800238736a3f94735af6faaa5d090fe1', 'nonce': b'R+\xb7\xa8;\xa2.\xf45{\xdd\xf7o\x12\x7f1J(K\xe7\xc0\x84\x11\xdb', 'signatures': [b'\n\xd3\x06\n\xb6\x06\n\x07Org1MSP\x12\xaa\x06-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMNU2FuIEZy\nYW5jaXNjbzEZMBcGA1UEChMQb3JnMS5leGFtcGxlLmNvbTEcMBoGA1UEAxMTY2Eu\nb3JnMS5leGFtcGxlLmNvbTAeFw0xODEwMTkwMzQ4MDBaFw0yODEwMTYwMzQ4MDBa\nMGwxCzAJBgNVBAYTAlVTMRMwEQYDVQQIEwpDYWxpZm9ybmlhMRYwFAYDVQQHEw1T\nYW4gRnJhbmNpc2NvMQ8wDQYDVQQLEwZjbGllbnQxHzAdBgNVBAMMFkFkbWluQG9y\nZzEuZXhhbXBsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAAR2zFPDBYIy\njZSXaB15ILW9MRUYkSiksD6Io+VBbsJex2S7Do7lJhYfpyg5Z1LEHLCQgHH/VJ1F\nMZmNHgIIHqWPo00wSzAOBgNVHQ8BAf8EBAMCB4AwDAYDVR0TAQH/BAIwADArBgNV\nHSMEJDAigCBrrGAV

True


### Join Peers into Channel, the response should be true if succeed

In [75]:
responses = await cli.channel_join(
                       requestor=org1_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

get genesis block successfully, block=data_hash: "\202\326D\347(\224\270\220\"\'H\\\300\275\000\027?\226=\270F\201\211\023\307\211J\334{\\\007p"



True


### For operations on peers from org2.example.com, org2_admin is required as requestor

In [76]:
responses = await cli.channel_join(
                       requestor=org2_admin,
                       channel_name='businesschannel',
                       peers=['peer0.org2.example.com',
                      'peer1.org2.example.com'],
                       orderer='orderer.example.com'
                       )

# The length of the response should be two as len(peers) = 2
print(len(responses) == 2)

get genesis block successfully, block=data_hash: "\202\326D\347(\224\270\220\"\'H\\\300\275\000\027?\226=\270F\201\211\023\307\211J\334{\\\007p"



True


## 3. Operate Chaincodes with Fabric Network

In [77]:
# This installs the example chaincode on the peers
# Make the client know there is a channel in the network
cli.new_channel('businesschannel')

#The GOPTAH settings is required for only running the example chaincode in the SDK
import os
gopath_bak = os.environ.get('GOPATH', '')
gopath = os.path.normpath(os.path.join(
                      os.path.dirname(os.path.realpath('__file__')),
                      'test/fixtures/chaincode'
                     ))

# The response should be true if succeed
responses = await cli.chaincode_install(
               requestor=org1_admin,
               peers=['peer0.org1.example.com',
                      'peer1.org1.example.com'],
               cc_path='github.com/example_cc',
               cc_name='example_cc',
               cc_version='v1.0'
               )
print(response==True)

New channel with name = businesschannel


True


In [87]:
# Instantiate Chaincode in Channel, the response should be true if succeed
args = ['a', '200', 'b', '300']

# This policy specifies the endorsement policy which is required while instantiating the chaincode
policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0},
        ]
    }
}
response = await cli.chaincode_instantiate(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is instantiated
               )
print(response==True)

False


In [86]:
# Invoke a chaincode
args = ['a', 'b', '100']
# The response should be true if succeed
response = await cli.chaincode_invoke(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               transient_map=None, # optional, for private data
               wait_for_event=True, # for being sure chaincode invocation has been commited in the ledger, default is on tx event
               #cc_pattern='^invoked*' # if you want to wait for chaincode event and you have a `stub.SetEvent("invoked", value)` in your chaincode
               )
print(response==True)

False


In [88]:
# Query a chaincode
args = ['b']
# The response should be true if succeed
response = await (cli.chaincode_query(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc'
               ))
print(response==True)

False


In [90]:
# Upgrade a chaincode
# policy, see https://hyperledger-fabric.readthedocs.io/en/release-1.4/endorsement-policies.html
policy = {
    'identities': [
        {'role': {'name': 'member', 'mspId': 'Org1MSP'}},
        {'role': {'name': 'admin', 'mspId': 'Org1MSP'}},
    ],
    'policy': {
        '1-of': [
            {'signed-by': 0}, {'signed-by': 1},
        ]
    }
}
response =     cli.chaincode_upgrade(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               args=args,
               cc_name='example_cc',
               cc_version='v1.0',
               cc_endorsement_policy=policy, # optional, but recommended
               collections_config=None, # optional, for private data policy
               transient_map=None, # optional, for private data
               wait_for_event=True # optional, for being sure chaincode is upgraded
               )
print(response==True)

False


#### Invoke and query the chaincode through the Gateway This has to be done after installing and instantiating the chaincode

In [92]:
import asyncio
from hfc.fabric_network.gateway import Gateway
from hfc.fabric_network.network import Network
from hfc.fabric_network.contract import Contract
from hfc.fabric import Client

loop = asyncio.get_event_loop()

cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user(org_name='org1.example.com', name='Admin')

new_gateway = Gateway() # Creates a new gateway instance
options = {'wallet': ''}
response = await new_gateway.connect('test/fixtures/network.json', options)
new_network = await new_gateway.get_network('businesschannel', org1_admin)
new_contract = new_network.get_contract('example_cc')
response = await new_contract.submit_transaction('businesschannel', ['a', 'b', '100'], org1_admin)
response  = await new_contract.evaluate_transaction('businesschannel', ['b'], org1_admin)

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])
Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])
<ipython-input-92-04a056cf2b13>:14: RuntimeWarning: coroutine 'Client.chaincode_upgrade' was never awaited
  response = await new_gateway.connect('test/fixtures/network.json', option

Exception: [version: 1
response {
  status: 200
  payload: "400"
}
payload: "\n \307\036\302c\377\220:\021!\026Z_ppvN\212\366)\353D\250\365\006\252\373!yd\035\232\354\022U\n5\022\027\n\nexample_cc\022\t\n\007\n\001b\022\002\010\002\022\032\n\004lscc\022\022\n\020\n\nexample_cc\022\002\010\001\032\010\010\310\001\032\003400\"\022\022\nexample_cc\032\004v1.0"
endorsement {
  endorser: "\n\007Org1MSP\022\252\006-----BEGIN CERTIFICATE-----\nMIICKDCCAc+gAwIBAgIRAK5JdC2UorbI3GYUD7Z+I08wCgYIKoZIzj0EAwIwczEL\nMAkGA1UEBhMCVVMxEzARBgNVBAgTCkNhbGlmb3JuaWExFjAUBgNVBAcTDVNhbiBG\ncmFuY2lzY28xGTAXBgNVBAoTEG9yZzEuZXhhbXBsZS5jb20xHDAaBgNVBAMTE2Nh\nLm9yZzEuZXhhbXBsZS5jb20wHhcNMTgxMDE5MDM0ODAwWhcNMjgxMDE2MDM0ODAw\nWjBqMQswCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5pYTEWMBQGA1UEBxMN\nU2FuIEZyYW5jaXNjbzENMAsGA1UECxMEcGVlcjEfMB0GA1UEAxMWcGVlcjAub3Jn\nMS5leGFtcGxlLmNvbTBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABF9ACNtWdD+C\nj5hejCA9qvYEO1pbizsTZzpixLZO96Hv7Bhjke0I4m92+m1QB3j3atPbIqYR1vI8\n5PsIwdaSnJejTTBLMA4GA1UdDwEB/wQ

### Get channel configuration

In [96]:
import asyncio
from hfc.fabric import Client

loop = asyncio.get_event_loop()
cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user('org1.example.com', 'Admin')

# Get channel config
response = cli.get_channel_config(
               requestor=org1_admin,
               channel_name='businesschannel',
               peers=['peer0.org1.example.com'],
               decode=True
               )
print(response)

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


<coroutine object Client.get_channel_config at 0x7fbb3ff98a40>


<ipython-input-96-5a3786d69021>:9: RuntimeWarning: coroutine 'Client.get_channel_config' was never awaited
  response = cli.get_channel_config(


### Use channel discovery

In [98]:
import asyncio
from hfc.fabric import Client

loop = asyncio.get_event_loop()
cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user('org1.example.com', 'Admin')

# Get config from local channel discovery
response = await cli.query_peers(
               requestor=org1_admin,
               peer='peer0.org1.example.com',
               channel='businesschannel',
               local=True,
               decode=True
               )

# Get config from channel discovery over the network
response = await cli.query_peers(
               requestor=org1_admin,
               peer='peer0.org1.example.com',
               channel='businesschannel',
               local=False,
               decode=True
               )

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])
New channel with name = discover-local
<ipython-input-98-677968f24fb4>:9: RuntimeWarning: coroutine 'Client.get_channel_config' was never awaited
  response = await cli.query_peers(
New channel with name = businesschannel
